# `RoBERTa` --> `Longformer`: build a "long" version of pretrained models

This notebook replicates the procedure descriped in the [Longformer paper](https://arxiv.org/abs/2004.05150) to train a Longformer model starting from the RoBERTa checkpoint. The same procedure can be applied to build the "long" version of other pretrained models as well.


In [ ]:
!pip install accelerate -U --quiet
!pip install datasets -U --quiet

In [ ]:
import logging
import os
import math
import copy
import pathlib
from pathlib import Path
from tqdm.auto import tqdm
import json

import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split

import torch
from dataclasses import dataclass, field

from datasets import Dataset
from transformers import RobertaTokenizerFast, DataCollatorForLanguageModeling, Trainer, AutoTokenizer
from transformers import RobertaForMaskedLM, AutoModel, AutoModelForMaskedLM, LongformerForMaskedLM
from transformers import TrainingArguments, HfArgumentParser
from transformers import LongformerSelfAttention

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
main_path = '/content/drive/MyDrive/datasets/KorCrawl'
main_path = Path(main_path)

### RobertaLong

`RobertaLongForMaskedLM` represents the "long" version of the `RoBERTa` model. It replaces `BertSelfAttention` with `RobertaLongSelfAttention`, which is a thin wrapper around `LongformerSelfAttention`.


In [ ]:
class RobertaLongSelfAttention(LongformerSelfAttention):
    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        output_attentions=False,
    ):
        return super().forward(hidden_states, attention_mask=attention_mask, output_attentions=output_attentions)


class RobertaLongForMaskedLM(RobertaForMaskedLM):
    def __init__(self, config):
        super().__init__(config)
        for i, layer in enumerate(self.roberta.encoder.layer):
            # replace the `modeling_bert.BertSelfAttention` object with `LongformerSelfAttention`
            layer.attention.self = RobertaLongSelfAttention(config, layer_id=i)

Starting from the `roberta-base` checkpoint, the following function converts it into an instance of `RobertaLong`. It makes the following changes:

- extend the position embeddings from `512` positions to `max_pos`. In Longformer, we set `max_pos=4096`

- initialize the additional position embeddings by copying the embeddings of the first `512` positions. This initialization is crucial for the model performance (check table 6 in [the paper](https://arxiv.org/pdf/2004.05150.pdf) for performance without this initialization)

- replaces `modeling_bert.BertSelfAttention` objects with `modeling_longformer.LongformerSelfAttention` with a attention window size `attention_window`

The output of this function works for long documents even without pretraining. Check tables 6 and 11 in [the paper](https://arxiv.org/pdf/2004.05150.pdf) to get a sense of the expected performance of this model before pretraining.

In [ ]:
def create_long_model(save_model_to, attention_window, max_pos):
    model = RobertaForMaskedLM.from_pretrained('klue/roberta-large')
    tokenizer = RobertaTokenizerFast.from_pretrained('klue/roberta-large', model_max_length=max_pos)
    config = model.config

    # extend position embeddings
    tokenizer.model_max_length = max_pos
    tokenizer.init_kwargs['model_max_length'] = max_pos
    current_max_pos, embed_size = model.roberta.embeddings.position_embeddings.weight.shape
    max_pos += 2  # NOTE: RoBERTa has positions 0,1 reserved, so embedding size is max position + 2
    config.max_position_embeddings = max_pos
    assert max_pos > current_max_pos
    # allocate a larger position embedding matrix
    new_pos_embed = model.roberta.embeddings.position_embeddings.weight.new_empty(max_pos, embed_size)
    # copy position embeddings over and over to initialize the new position embeddings
    k = 2
    step = current_max_pos - 2
    while k < max_pos - 1:
        new_pos_embed[k:(k + step)] = model.roberta.embeddings.position_embeddings.weight[2:]
        k += step
    model.roberta.embeddings.position_embeddings.weight.data = new_pos_embed
    model.roberta.embeddings.position_ids.data = torch.tensor([i for i in range(max_pos)]).reshape(1, max_pos)

    # replace the `modeling_bert.BertSelfAttention` object with `LongformerSelfAttention`
    config.attention_window = [attention_window] * config.num_hidden_layers
    for i, layer in enumerate(model.roberta.encoder.layer):
        longformer_self_attn = LongformerSelfAttention(config, layer_id=i)
        longformer_self_attn.query = layer.attention.self.query
        longformer_self_attn.key = layer.attention.self.key
        longformer_self_attn.value = layer.attention.self.value

        longformer_self_attn.query_global = copy.deepcopy(layer.attention.self.query)
        longformer_self_attn.key_global = copy.deepcopy(layer.attention.self.key)
        longformer_self_attn.value_global = copy.deepcopy(layer.attention.self.value)

        layer.attention.self = longformer_self_attn

    logger.info(f'saving model to {save_model_to}')
    model.save_pretrained(save_model_to)
    tokenizer.save_pretrained(save_model_to)
    return model, tokenizer

Pretraining on Masked Language Modeling (MLM) doesn't update the global projection layers. After pretraining, the following function copies `query`, `key`, `value` to their global counterpart projection matrices.
For more explanation on "local" vs. "global" attention, please refer to the documentation [here](https://huggingface.co/transformers/model_doc/longformer.html#longformer-self-attention).

In [ ]:
def copy_proj_layers(model):
    for i, layer in enumerate(model.roberta.encoder.layer):
        layer.attention.self.query_global = copy.deepcopy(layer.attention.self.query)
        layer.attention.self.key_global = copy.deepcopy(layer.attention.self.key)
        layer.attention.self.value_global = copy.deepcopy(layer.attention.self.value)
    return model

### Pretrain and Evaluate on masked language modeling (MLM)

The following function pretrains and evaluates a model on MLM.

**Training hyperparameters**

- Following RoBERTa pretraining setting, we set number of tokens per batch to be `2^18` tokens. Changing this number might require changes in the lr, lr-scheudler, #steps and #warmup steps. Therefor, it is a good idea to keep this number constant.

- Note that: `#tokens/batch = batch_size x #gpus x gradient_accumulation x seqlen`
   
- In [the paper](https://arxiv.org/pdf/2004.05150.pdf), we train for 65k steps, but 3k is probably enough (check table 6)

- **Important note**: The lr-scheduler in [the paper](https://arxiv.org/pdf/2004.05150.pdf) is polynomial_decay with power 3 over 65k steps. To train for 3k steps, use a constant lr-scheduler (after warmup). Both lr-scheduler are not supported in HF trainer, and at least **constant lr-scheduler** will need to be added.

- Pretraining will take 2 days on 1 x 32GB GPU with fp32. Consider using fp16 and using more gpus to train faster (if you increase `#gpus`, reduce `gradient_accumulation` to maintain `#tokens/batch` as mentioned earlier).

- As a demonstration, this notebook is training on wikitext103 but wikitext103 is rather small that it takes 7 epochs to train for 3k steps Consider doing a single epoch on a larger dataset (800M tokens) instead.

- Set #gpus using `CUDA_VISIBLE_DEVICES`

In [ ]:
@dataclass
class ModelArgs:
    attention_window: int = field(default=512, metadata={"help": "Size of attention window"})
    max_pos: int = field(default=4096, metadata={"help": "Maximum position"})

parser = HfArgumentParser((TrainingArguments, ModelArgs,))

training_args, model_args = parser.parse_args_into_dataclasses(look_for_args_file=False, args=[
    '--output_dir', '/content/drive/MyDrive/projects/ExpertLLM/model/',
    '--warmup_steps', '150',
    '--learning_rate', '0.0003',
    '--weight_decay', '0.01',
    '--adam_epsilon', '1e-6',
    '--max_steps', '700',
    '--logging_steps', '3',
    #'--save_steps', '500',
    '--max_grad_norm', '5.0',
    '--per_device_eval_batch_size', '8',
    '--per_device_train_batch_size', '1',  # 32GB gpu with fp32
    '--gradient_accumulation_steps', '32',
    #'--evaluate_during_training',
    '--do_train',
    '--do_eval',
])
training_args.val_datapath = main_path / 'sample_train_4000.csv'
training_args.train_datapath = main_path / 'sample_val_4000.csv'

# Choose GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### Put it all together

1) Evaluating `roberta-base` on MLM to establish a baseline. Validation `bpc` = `2.536` which is higher than the `bpc` values in table 6 [here](https://arxiv.org/pdf/2004.05150.pdf) because wikitext103 is harder than our pretraining corpus.

In [ ]:
roberta_base = RobertaForMaskedLM.from_pretrained('klue/roberta-large')
roberta_base_tokenizer = RobertaTokenizerFast.from_pretrained('klue/roberta-large')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


2) As descriped in `create_long_model`, convert a `roberta-base` model into `roberta-base-4096` which is an instance of `RobertaLong`, then save it to the disk.

In [ ]:
model_path = f'{training_args.output_dir}/roberta-large-{model_args.max_pos}'
if not os.path.exists(model_path):
    os.makedirs(model_path)

logger.info(f'Converting roberta-large into roberta-large-{model_args.max_pos}')
model, tokenizer = create_long_model(
    save_model_to=model_path, attention_window=model_args.attention_window, max_pos=model_args.max_pos)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


3) Load `roberta-base-4096` from the disk. This model works for long sequences even without pretraining. If you don't want to pretrain, you can stop here and start finetuning your `roberta-base-4096` on downstream tasks 🎉🎉🎉

In [ ]:
logger.info(f'Loading the model from {model_path}')
tokenizer = RobertaTokenizerFast.from_pretrained(model_path)
model = RobertaLongForMaskedLM.from_pretrained(model_path)

4) Pretrain `roberta-base-4096` for `3k` steps, each steps has `2^18` tokens. Notes:

- The `training_args.max_steps = 3 ` is just for the demo. **Remove this line for the actual training**

- Training for `3k` steps will take 2 days on a single 32GB gpu with `fp32`. Consider using `fp16` and more gpus to train faster.

- Tokenizing the training data the first time is going to take 5-10 minutes.

- MLM validation `bpc` **before** pretraining: **2.652**, a bit worse than the **2.536** of `roberta-base`. As discussed in [the paper](https://arxiv.org/pdf/2004.05150.pdf) this is expected because the model didn't learn yet to work with the sliding window attention.

- MLM validation `bpc` after pretraining for a few number of steps: **2.628**. It is quickly getting better. By 3k steps, it should be better than the **2.536** of `roberta-base`.

5) Copy global projection layers. MLM pretraining doesn't train global projections, so we need to call `copy_proj_layers` to copy the local projection layers to the global ones.

In [ ]:
logger.info(f'Copying local projection layers into global projection layers ... ')
model = copy_proj_layers(model)
logger.info(f'Saving model to {model_path}')
model.save_pretrained(model_path)

In [ ]:
logger.info(f'Loading the model from {model_path}')
tokenizer = RobertaTokenizerFast.from_pretrained(model_path)
model = RobertaLongForMaskedLM.from_pretrained(model_path)

In [ ]:
model

RobertaLongForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=1)
      (position_embeddings): Embedding(4098, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaLongSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (query_global): Linear(in_features=1024, out_features=1024, bias=True)
              (key_global): Linear(in_features=1024, out_features=1024, bias=True)
              (value_globa

In [ ]:
longformer = LongformerForMaskedLM.from_pretrained('allenai/longformer-large-4096')
longformer

LongformerForMaskedLM(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(4098, 1024, padding_idx=1)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0-23): 24 x LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (query_global): Linear(in_features=1024, out_features=1024, bias=True)
              (key_global): Linear(in_features=1024, out_features=1024, bias=True)
          

In [ ]:
def to_longformer(model):
    new_model = LongformerForMaskedLM.from_pretrained('allenai/longformer-large-4096')
    new_model.longformer.embeddings.word_embeddings = model.roberta.embeddings.word_embeddings
    new_model.longformer.embeddings.token_type_embeddings = model.roberta.embeddings.token_type_embeddings
    new_model.longformer.embeddings.LayerNorm = model.roberta.embeddings.LayerNorm
    new_model.longformer.embeddings.position_embeddings = model.roberta.embeddings.position_embeddings
    for i, layer in enumerate(new_model.longformer.encoder.layer):
        layer.attention.self.query = model.roberta.encoder.layer[i].attention.self.query
        layer.attention.self.query_global = model.roberta.encoder.layer[i].attention.self.query_global
        layer.attention.self.key = model.roberta.encoder.layer[i].attention.self.key
        layer.attention.self.key_global = model.roberta.encoder.layer[i].attention.self.key_global
        layer.attention.self.value = model.roberta.encoder.layer[i].attention.self.value
        layer.attention.self.value_global = model.roberta.encoder.layer[i].attention.self.value_global
        layer.attention.output.dense = model.roberta.encoder.layer[i].attention.output.dense
        layer.attention.output.LayerNorm = model.roberta.encoder.layer[i].attention.output.LayerNorm
        layer.intermediate.dense = model.roberta.encoder.layer[i].intermediate.dense
        layer.output.dense = model.roberta.encoder.layer[i].output.dense
        layer.output.LayerNorm = model.roberta.encoder.layer[i].output.LayerNorm
    new_model.config.vocab_size = model.config.vocab_size
    new_model.config.hidden_size = model.config.hidden_size
    new_model.config.num_hidden_layers = model.config.num_hidden_layers
    new_model.config.num_attention_heads = model.config.num_attention_heads
    new_model.config.type_vocab_size = model.config.type_vocab_size
    new_model.config.max_position_embeddings = model.config.max_position_embeddings
    new_model.config.attention_window = model.config.attention_window

    new_model.lm_head.dense = model.lm_head.dense
    new_model.lm_head.layer_norm = model.lm_head.layer_norm
    new_model.lm_head.decoder = model.lm_head.decoder
    return new_model

In [ ]:
longfomer_model = to_longformer(model)
longfomer_model

LongformerForMaskedLM(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(4098, 1024, padding_idx=1)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0-23): 24 x LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (query_global): Linear(in_features=1024, out_features=1024, bias=True)
              (key_global): Linear(in_features=1024, out_features=1024, bias=True)
          

In [ ]:
dataset = Dataset.from_pandas(pd.read_csv(training_args.train_datapath))
dataset

Dataset({
    features: ['text'],
    num_rows: 75042
})

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [ ]:
dataset = dataset.map(lambda examples: tokenizer(examples["text"], padding="max_length", truncation=True), batched=True)

Map:   0%|          | 0/75042 [00:00<?, ? examples/s]

In [ ]:
trainer = Trainer(
    model=longfomer_model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
len(dataset[0]['input_ids'])

4096

In [ ]:
trainer.train()

Step,Training Loss
3,4.539300
6,4.148200
9,2.560900
12,1.675000
15,1.523800
18,1.426100
21,1.340200
24,1.150800
27,1.228500
30,1.172600


TrainOutput(global_step=700, training_loss=1.4610219935008457, metrics={'train_runtime': 29773.7298, 'train_samples_per_second': 0.752, 'train_steps_per_second': 0.024, 'total_flos': 2.086495132581888e+17, 'train_loss': 1.4610219935008457, 'epoch': 0.2984995069427787})

In [ ]:
trainer.save_model(f'{training_args.output_dir}/longformer-large-4096-800')